In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('kaggle-data/dataset.csv')
df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
one_hots = [df['Disease']]
for i in range(1, 18):
    one_hots.append(pd.get_dummies(df.iloc[:, i]))
    one_hots[-1].columns = one_hots[-1].columns.str.replace(' ', '')
    one_hots[-1].columns = one_hots[-1].columns.str.replace('_', ' ')
df = pd.concat(one_hots, axis=1)
for column in df:
    if isinstance(df[column], pd.Series):
        continue
    merged = df[column].sum(axis=1)
    df = df.drop(columns=column)
    df[column] = merged
df.head()

,Disease,continuous sneezing,muscle wasting,itching,acidity,back pain,bladder discomfort,breathlessness,burning micturition,chest pain,...,rusty sputum,coma,palpitations,receiving unsterile injections,sinus pressure,runny nose,stomach bleeding,congestion,blood in sputum,loss of smell
0,Fungal infection,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from sklearn import tree

impact_factors = []
trees = {}
for entry in df.columns[1:]:
    X = df.loc[df[entry] == 1]
    y = X['Disease']
    X = X.drop(columns=['Disease', entry])
    clf = tree.DecisionTreeClassifier(criterion='entropy', min_samples_leaf=0.05)
    clf.fit(X, y)
    trees[entry] = tree.export_text(clf, feature_names=X.columns.to_list())
    impact_factors.append((entry, clf.get_n_leaves()))
impact_factors = pd.DataFrame(impact_factors, columns=['Symptons', 'Leaves']).sort_values('Leaves', ignore_index=True)
impact_factors

,Symptons,Leaves
0,movement stiffness,1
1,swollen blood vessels,1
2,mucoid sputum,1
3,internal itching,1
4,inflammatory nails,1
...,...,...
126,nausea,10
127,loss of appetite,10
128,high fever,12
129,vomiting,17


In [4]:
entries = list(impact_factors.to_dict()['Symptons'].values())[-5:]

trees = [ trees[entry] for entry in entries]

In [5]:
print(trees[0])

|--- yellowing of eyes <= 0.50
|   |--- muscle pain <= 0.50
|   |   |--- chills <= 0.50
|   |   |   |--- palpitations <= 0.50
|   |   |   |   |--- class: (vertigo) Paroymsal  Positional Vertigo
|   |   |   |--- palpitations >  0.50
|   |   |   |   |--- class: Hypoglycemia
|   |   |--- chills >  0.50
|   |   |   |--- class: Typhoid
|   |--- muscle pain >  0.50
|   |   |--- pain behind the eyes <= 0.50
|   |   |   |--- class: Malaria
|   |   |--- pain behind the eyes >  0.50
|   |   |   |--- class: Dengue
|--- yellowing of eyes >  0.50
|   |--- fatigue <= 0.50
|   |   |--- mild fever <= 0.50
|   |   |   |--- class: Chronic cholestasis
|   |   |--- mild fever >  0.50
|   |   |   |--- class: hepatitis A
|   |--- fatigue >  0.50
|   |   |--- coma <= 0.50
|   |   |   |--- vomiting <= 0.50
|   |   |   |   |--- class: Hepatitis C
|   |   |   |--- vomiting >  0.50
|   |   |   |   |--- class: Hepatitis D
|   |   |--- coma >  0.50
|   |   |   |--- class: Hepatitis E



In [6]:
import json

def recursive():
  global i
  line = nodes[i]
  i += 1
  if 'class' in line:
    return { 'type': 'result' }
  else:
    feature = line.split('- ')[1].split(' >')[0].split(' <')[0]
    if nodes[i].count('|') > line.count('|'):
      left = recursive()
    if nodes[i].count('|') == line.count('|'):
      i += 1
    if nodes[i].count('|') > line.count('|'):
      right = recursive()
    if '<=' in line: # swap Y/N
      tmp = left
      left = right
      right = tmp
    return { 'type': 'question', 'feature': feature, 'Yes' : left, 'No' : right }

flow = { 'type': 'entries', 'entires': {}}
for idx, entry in enumerate(entries):
  i = 0
  nodes = trees[i].split('\n')
  flow['entires'][entry] = recursive()
flow

{'type': 'entries',
 'entires': {'nausea': {'type': 'question',
   'feature': 'yellowing of eyes',
   'Yes': {'type': 'question',
    'feature': 'fatigue',
    'Yes': {'type': 'question',
     'feature': 'coma',
     'Yes': {'type': 'result'},
     'No': {'type': 'question',
      'feature': 'vomiting',
      'Yes': {'type': 'result'},
      'No': {'type': 'result'}}},
    'No': {'type': 'question',
     'feature': 'mild fever',
     'Yes': {'type': 'result'},
     'No': {'type': 'result'}}},
   'No': {'type': 'question',
    'feature': 'muscle pain',
    'Yes': {'type': 'question',
     'feature': 'pain behind the eyes',
     'Yes': {'type': 'result'},
     'No': {'type': 'result'}},
    'No': {'type': 'question',
     'feature': 'chills',
     'Yes': {'type': 'result'},
     'No': {'type': 'question',
      'feature': 'palpitations',
      'Yes': {'type': 'result'},
      'No': {'type': 'result'}}}}},
  'loss of appetite': {'type': 'question',
   'feature': 'yellowing of eyes',
   'Y

In [7]:
severity = pd.read_csv('kaggle-data/Symptom-severity.csv')
severity.index = severity['Symptom'].str.replace('_', ' ')
severity = severity.drop(columns='Symptom')
severity = severity.to_dict()['weight']

In [8]:
empty_dict = 0
def generateResult(node, params, level):
    global empty_dict
    if node['type'] == 'result':
        sample = df
        for param in params:
          sample = sample.loc[sample[param[0]] == param[1]]
        node['severity'] = level
        node['probability'] = (sample['Disease'].value_counts() / sample.shape[0]).to_dict()
        print(params)
        print(node)
        if node['probability'] == dict():
          empty_dict += 1
    if node['type'] == 'question':
        generateResult(node['Yes'], params+[(node['feature'], 1)], level+severity[node['feature']])
        generateResult(node['No'], params+[(node['feature'], 0)], level)

for entry in entries:
  i = 0
  generateResult(flow['entires'][entry], [(entry, 1)], severity[entry])
print('Empty prob', empty_dict)

with open('./bot/flow.json', 'w') as f:
  json.dump(flow, f, indent=2)

[('nausea', 1), ('yellowing of eyes', 1), ('fatigue', 1), ('coma', 1)]
{'type': 'result', 'severity': 20, 'probability': {'Hepatitis E': 1.0}}
[('nausea', 1), ('yellowing of eyes', 1), ('fatigue', 1), ('coma', 0), ('vomiting', 1)]
{'type': 'result', 'severity': 18, 'probability': {'Hepatitis D': 1.0}}
[('nausea', 1), ('yellowing of eyes', 1), ('fatigue', 1), ('coma', 0), ('vomiting', 0)]
{'type': 'result', 'severity': 13, 'probability': {'Hepatitis C': 0.9411764705882353, 'Hepatitis D': 0.058823529411764705}}
[('nausea', 1), ('yellowing of eyes', 1), ('fatigue', 0), ('mild fever', 1)]
{'type': 'result', 'severity': 14, 'probability': {'hepatitis A': 1.0}}
[('nausea', 1), ('yellowing of eyes', 1), ('fatigue', 0), ('mild fever', 0)]
{'type': 'result', 'severity': 9, 'probability': {'Chronic cholestasis': 0.8571428571428571, 'Hepatitis C': 0.047619047619047616, 'Hepatitis D': 0.047619047619047616, 'Hepatitis E': 0.047619047619047616}}
[('nausea', 1), ('yellowing of eyes', 0), ('muscle pai